# Generating an RDF graph with ISA

## Abstract

The aim of this notebook is to:
   - learn how to serialize an ISA investigation in JSON-LD with different contexts
   - generate an RDF graph from that JSON-LD
   - execute a sparQL query on that graph.

## 1. Getting the tools

In [1]:
# Let's first import all the packages we need

In [2]:
from os import path
import json

from rdflib import Graph, Namespace

from isatools.isajson import load
from isatools.model import set_context

## 2. Reading and loading an ISA Investigation in memory from an ISA-JSON instance

In [3]:
filepath = path.join('json', 'BII-S-3', 'BII-S-3.json')
with open(filepath, 'r') as f:
    investigation = load(f)

## 3. Setting options for the contexts binding

In [4]:
set_context(new_context='wdt', combine=False, local=False)

The `set_context()` method takes three parameters:
    - new_context: to choose the vocabulary to use between ``sdo``, ``obo`` and ``wdt``
    - combine: if ``True``, only one context will be used
    - local: if ``True``, uses local files else the GitHub contexts

## 4. Generate a JSON-LD serialization

In [5]:
ld = investigation.to_dict(ld=True)

The investigation can be serialized to json with the `to_dict()` method. By passing the optional parameter `ld=True`, the serializer binds the `@type`, `@context` and `@id` to each object in the JSON.

## 5. Generate an RDF graph

Before we can generate a graph we need to create the proper namespaces and transform the `ld` variable into a string

In [6]:
# Creating the namespace
WDT = Namespace("http://www.wikidata.org/wiki/")
WDTP = Namespace('https://www.wikidata.org/wiki/Property:')
ISA = Namespace('https://isa.org/')

ld_string = json.dumps(ld) # Get a string representation of the ld variable
graph = Graph() # Create an empty graph
graph.parse(data=ld_string, format='json-ld') # Load the data into the graph

# Finally, bind the namespaces to the graph
graph.bind('wdt', WDT)
graph.bind('isa', ISA)
graph.bind('wdtp', WDTP)

2022-10-06 08:21:44,104 [WARNING]: term.py(_castLexicalToPython:2086) >> Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x000001C74B85E830>
Traceback (most recent call last):
  File "F:\Work\isa-api\venv3.10\lib\site-packages\rdflib\term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "F:\Work\isa-api\venv3.10\lib\site-packages\isodate\isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 date format: ''
2022-10-06 08:21:44,106 [WARNING]: term.py(_castLexicalToPython:2086) >> Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x000001C74B85E830>
Traceback (most recent call last):
  File "F:\Work\isa-api\venv3.10\lib\site-packages\rdflib\term.py", line 2084, i

## 6. Create a small sparQL query and execute it

In [8]:
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX wdtp: <https://www.wikidata.org/wiki/Property:>
PREFIX wdt: <http://www.wikidata.org/wiki/>

SELECT distinct ?protocolTypeName
WHERE {
    ?p rdf:type wdt:Q41689629 . # Is a protocol
    ?p wdtp:P7793 ?protocolType .
    ?protocolType wdtp:P527 ?protocolTypeName . # Get each protocol type name
    FILTER (?protocolTypeName!=""^^wdt:Q1417099) # Filter out empty protocol type name
}
"""

for node in graph.query(query):
    print(node.asdict())

{'protocolTypeName': rdflib.term.Literal('sample collection', datatype=rdflib.term.URIRef('rdfs:label'))}
{'protocolTypeName': rdflib.term.Literal('nucleic acid extraction', datatype=rdflib.term.URIRef('rdfs:label'))}
{'protocolTypeName': rdflib.term.Literal('reverse transcription', datatype=rdflib.term.URIRef('rdfs:label'))}
{'protocolTypeName': rdflib.term.Literal('library construction', datatype=rdflib.term.URIRef('rdfs:label'))}
{'protocolTypeName': rdflib.term.Literal('nucleic acid sequencing', datatype=rdflib.term.URIRef('rdfs:label'))}
{'protocolTypeName': rdflib.term.Literal('data transformation', datatype=rdflib.term.URIRef('rdfs:label'))}
